In [21]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [25]:
perf_url = 'http://www.operatoday.com/content/reviews/performances/index.php?page='
reviews=[]

In [ ]:
for page in range(1,76):

    if page%5==0:
        print(page)
        
    url      = perf_url + str(page)
    r        = requests.get(url)
    soup     = BeautifulSoup(r.text, "html5lib")
    listings = soup.find(attrs={"id": "subfeature"})
    listings = listings.find_all(attrs={"class": "story"})

    for listing in listings:
        
        review  = []
        date    = listing.find('h5').get_text()
        date    = datetime.strptime(date, '%d %b %Y').date() 
        title = listing.find('a').get_text()
        rev_url = listing.find('a').get('href')        
        try:
            work = listing.find('a').find('em').get_text().strip()          
        except:
            work = ''

        r    = requests.get(rev_url)
        soup = BeautifulSoup(r.text, "html5lib")
        if 'not found' in soup.title.get_text().lower():
            break
            
        if work=='':
            work = soup.find(attrs={"id": "relatedproduct"})
            try:
                work = work.find('strong').get_text().strip()
            except:
                try:
                    work = work.find('p').get_text().strip()
                except:
                    work = ''
        
        text = soup.find_all(attrs={"id": "article"}) + soup.find(attrs={"class": "story"}).find_all('p')        
        full_text=''
        for w in text:
            if w.get('class')==None:
                full_text += w.get_text()        

        review.append(work)                
        review.append(date)
        review.append(title)
        review.append(rev_url)  
        review.append(full_text)        
        reviews.append(review)
        
import pandas as pd                     
headers = ['work', 'date', 'title', 'url', 'full_text']
df = pd.DataFrame(reviews, columns=headers)
df.to_csv('../../data/processed/reviews/operatoday.csv')